# Proof to minimum number of active sboxes in a MixColumn in 9

# Importing necessary libraries

In [1]:
import random
import copy
from collections import deque
from IPython.core.display import HTML , Math
import galois
GF = galois.GF(2**4)
print(GF.properties)

Galois Field:
  name: GF(2^4)
  characteristic: 2
  degree: 4
  order: 16
  irreducible_poly: x^4 + x + 1
  is_primitive_poly: True
  primitive_element: x


# PHOTON<sub>256</sub>- Permutation function

In [2]:
# ---------------------------------------- PHOTON-256-PERMUTATION ---------------------------------------------
## SBOX LIST
sbox_list = [0xc,5,6,0xb,9,0,0xa,0xd,3,0xe,0xf,8,4,7,1,2]
def list_64_to_8x8_matrix(s) -> list[list[int]]:
    assert len(s) == 64
    m = [[s[i+(j*8)] for j in range(8)] for i in range(8)]
    return m
def matrix_8x8_to_hex_list(m : list[list[int]]):
    lst = []
    for col in range(8):
        for row in range(8):
            lst.append(m[row][col])
    return lst
## 1.ADD-CONSTANT
def add_constant(X : list,k : list):
    new_X = copy.deepcopy(X)
    RC = [1, 3, 7, 14, 13, 11, 6, 12, 9, 2, 5, 10]
    IC = [0, 1, 3, 7, 15, 14, 12, 8]
    for i in range(8):
        new_X[i][0] = new_X[i][0] ^ RC[k] ^ IC[i]
    return new_X
## 2.SUB-CELL
def sub_cell(X):
    new_X = copy.deepcopy(X)
    for i in range(8):
        for j in range(8):
            new_X[i][j] = sbox_list[new_X[i][j]]
    return new_X
## 3.SHIFT-ROW
def shift_row(X):
    new_X = copy.deepcopy(X)
    for i in range(8):
        temp = deque(new_X[i])
        temp.rotate(-1*i)
        new_X[i] = list(temp)
    return new_X
# def shift_row(X):
#     new_X = copy.deepcopy(X)
#     for i in range(8):
#         temp = deque(new_X[i])
#         temp.rotate(1*i)
#         new_X[i] = list(temp)
#     return new_X
## 4.MIX-COLUMN-SERIAL
def serial(lst):
    M = []
    for i in range(7):
        a = [0 for j in range(8)]
        a[i+1] = 1
        M.append(a)
    M.append(copy.deepcopy(lst))
    return M
def matrix_mul(m1,m2):
    new_m = [[0 for j in range(8)] for i in range(8)]
    for i in range(8):
        for j in range(8):
            s = 0
            for temp in range(8):
                s ^= int(GF(m1[i][temp]) * GF(m2[temp][j]))
            new_m[i][j] = s
    return new_m
def mix_column_serial(X):
    new_X = copy.deepcopy(X)
    M = serial([2, 4, 2, 11, 2, 8, 5, 6])
    M8 = matrix_mul(M,M)
    for i in range(6):
        M8 = matrix_mul(M8,M)
    new_X = matrix_mul(M8,new_X)
    return new_X
## PHOTON 256 PERMUTATION FUNCTION
def PHOTON_256(input_hex_str = "0123456789abcdef0123456789abcdef0123456789abcdef0123456789abcdef"):
    if type(input_hex_str) == "":
        input_hex_str = [int(i,16) for i in input_hex_str]
    assert len(input_hex_str) == 64
    X = list_64_to_8x8_matrix(input_hex_str)
    # 0 to 11
    for i in range(12):
        X = add_constant(X,i)
        X = sub_cell(X)
        X = shift_row(X)
        X = mix_column_serial(X)
    X = matrix_8x8_to_hex_list(X)
    assert len(X) == 64
    return X

# Utility Functions

In [3]:
def flip_boxes(m,box_nums=[]):
    new_m = copy.deepcopy(m)
    for pos in box_nums:
        x = (pos-1) % 8
        y = (pos-1) // 8
        new_m[x][y] = new_m[x][y] ^ random.randint(1,15)
    return new_m
def matrix_xor(m1,m2):
    l = len(m1[0])
    new_m = [[0 for i in range(l)] for j in range(l)]
    for i in range(l):
        for j in range(l):
            new_m[i][j] = m1[i][j] ^ m2[i][j]
    return new_m
def get_num_of_active_boxes(m):
    l = len(m)
    count = 0
    for i in range(l):
        for j in range(l):
            if m[i][j] > 0:
                count += 1
    return count

# Main implementation 

In [4]:
temp_data = []

In [34]:
for i1 in range(2):
    for i2 in range(2):
        for i3 in range(2):
            for i4 in range(2):
                for i5 in range(2):
                    for i6 in range(2):
                        for i7 in range(2):
                            for i8 in range(2):
                                lst1 = [i1,i2,i3,i4,i5,i6,i7,i8]
                                lst2 = []
                                for i in range(8):
                                    if lst1[i] == 1:
                                        lst2.append(i+1)
                                msg_a = [[random.randint(0,15) for i in range(8)] for j in range(8)]
                                msg_b = flip_boxes(msg_a,lst2)
                                in_diff_m = matrix_xor(msg_a,msg_b)
                                in_num = get_num_of_active_boxes(in_diff_m)

                                msg_out_a = mix_column_serial(msg_a)
                                msg_out_b = mix_column_serial(msg_b)
                                out_diff_m = matrix_xor(msg_out_a,msg_out_b)
                                out_num = get_num_of_active_boxes(out_diff_m)
                                temp_data.append([in_num,out_num,in_num+out_num])
                                print(in_num,out_num,in_num+out_num)

0 0 0
1 8 9
1 8 9
2 8 10
1 8 9
2 7 9
2 7 9
3 8 11
1 8 9
2 8 10
2 7 9
3 8 11
2 8 10
3 8 11
3 8 11
4 7 11
1 8 9
2 7 9
2 7 9
2 8 10
2 8 10
2 7 9
3 7 10
4 8 12
2 7 9
3 6 9
3 7 10
4 8 12
2 7 9
4 8 12
4 8 12
5 8 13
1 8 9
2 8 10
2 8 10
3 7 10
1 8 9
3 8 11
3 8 11
4 8 12
2 8 10
3 7 10
3 8 11
4 7 11
3 8 11
4 8 12
4 8 12
5 7 12
2 7 9
2 8 10
3 8 11
4 8 12
3 8 11
3 7 10
4 8 12
5 8 13
3 8 11
4 8 12
4 6 10
5 7 12
4 8 12
5 7 12
5 8 13
5 6 11
1 8 9
2 7 9
2 7 9
3 8 11
1 8 9
2 7 9
3 8 11
4 7 11
2 7 9
3 8 11
2 8 10
4 8 12
3 7 10
4 8 12
4 7 11
5 8 13
2 8 10
3 7 10
2 8 10
4 8 12
3 7 10
4 7 11
4 7 11
5 7 12
3 8 11
4 7 11
4 7 11
5 8 13
4 8 12
4 8 12
5 7 12
5 7 12
2 8 10
2 7 9
3 7 10
3 7 10
3 8 11
4 8 12
4 8 12
5 8 13
3 8 11
4 7 11
4 8 12
4 8 12
4 7 11
5 7 12
4 8 12
6 8 14
3 6 9
4 7 11
3 8 11
5 8 13
4 8 12
5 7 12
5 7 12
6 8 14
3 8 11
4 8 12
5 8 13
5 8 13
5 8 13
6 8 14
6 7 13
5 7 12
1 8 9
2 7 9
2 8 10
3 7 10
1 8 9
3 6 9
3 8 11
4 8 12
2 7 9
3 6 9
3 8 11
4 8 12
3 8 11
4 8 12
3 7 10
5 8 13
2 7 9
3 8 11
3 8 11
4 8 

In [5]:
temp_data =[]
count = 0
while True:
    count += 1
#     lst1 = [1,1,1,1, 1,1,1,1]
    lst2 = [1,2,3,4,5,6,7,8,]
#     for i in range(8):
#         if lst1[i] == 1:
#             lst2.append(i+1)
    msg_a = [[random.randint(0,15) for i in range(8)] for j in range(8)]
    msg_b = flip_boxes(msg_a,lst2)
    in_diff_m = matrix_xor(msg_a,msg_b)
    in_num = get_num_of_active_boxes(in_diff_m)

    msg_out_a = mix_column_serial(msg_a)
    msg_out_b = mix_column_serial(msg_b)
    out_diff_m = matrix_xor(msg_out_a,msg_out_b)
    out_num = get_num_of_active_boxes(out_diff_m)
    temp_data.append([in_num,out_num,in_num+out_num])
    if out_num == 1:
        break
    if out_num <= 6:
        print(count,"-",out_num)
    

8 - 5
11 - 6
16 - 6
17 - 6
23 - 5
44 - 6
55 - 4
63 - 6
65 - 6
102 - 6
103 - 6
118 - 6
130 - 6
134 - 6
147 - 6
151 - 6
163 - 5
164 - 6
184 - 6
189 - 6
193 - 6
228 - 6
232 - 6
253 - 6
259 - 6
274 - 6
278 - 6
304 - 6
305 - 6
360 - 4
361 - 6
362 - 6
369 - 6
371 - 6
376 - 6
383 - 6
390 - 6
394 - 6
405 - 6
415 - 6
449 - 6
464 - 6
467 - 6
490 - 6
492 - 6
496 - 6
524 - 5
542 - 6
567 - 6
580 - 6
590 - 5
598 - 6
599 - 6
610 - 5
617 - 6
618 - 6
627 - 6
637 - 5
684 - 5
691 - 6
692 - 5
699 - 6
707 - 6
723 - 6
746 - 6
757 - 6
773 - 6
783 - 6
800 - 6
801 - 6
810 - 6
814 - 6
824 - 6
854 - 6
865 - 6
878 - 6
909 - 6
910 - 6
930 - 6
957 - 6
966 - 6
973 - 6
1003 - 6
1016 - 6
1017 - 6
1032 - 6
1072 - 6
1074 - 5
1104 - 5
1145 - 6
1151 - 6
1164 - 6
1174 - 6
1200 - 6
1211 - 6
1213 - 6
1231 - 6
1267 - 6
1285 - 6
1295 - 6
1296 - 6
1335 - 6
1348 - 5
1351 - 6
1359 - 6
1375 - 6
1376 - 6
1388 - 6
1392 - 6
1413 - 6
1421 - 6
1427 - 5
1445 - 6
1453 - 6
1491 - 6
1515 - 6
1526 - 5
1550 - 6
1556 - 6
1558 - 6
1574 - 6
157

KeyboardInterrupt: 

### Conclusion
From this we can conclude that there are minimum of 9 active elements in __Mix-Column__ step

# - - - END - - -